In [1]:
from genai import gemini
from inspect import getsource
from smells import smells
import subprocess
import sys

# select model to generate the unit tests

In [2]:
m = gemini.GeminiModel()

2025-03-30 19:11:00.171 | INFO     | genai.gemini:__init__:22 - using gemini-2.0-pro-exp-02-05, temp 1.0, top_p 1, max_output_tokens 2048


# define the package and function for which to generate test

- it must be importable.
- we use funcs in file `misc.py`

In [3]:
module_name = 'misc'
#function_name = 'isprime'
#function_name = 'cauchy'
function_name = 'change_money'

# inspect function to test

In [34]:
exec(f'from modules.{module_name} import {function_name}')
function = eval(function_name)
print (getsource(function))

def change_money(n):
    """
    given n, a number of cents between 1 and 99
    computes the number of coins of 1c, 10c and 25c to reach n
    """
    change=[0,0,0]
    if (n<1 or n>99):
        return None
    
    else:
        change[2]=n//25
        n-=change[2]*25
        change[1]=n//10
        change[0]=n%10
        return change



# create generation prompt

In [5]:
prompt = f"""
generate a unit test for this python function which is accessible with the following import

from modules.{module_name} import {function_name}

<FUNCTION_DEFINITION>
{getsource(function)}
</FUNCTION_DEFINITION>

- enclose the unit test class tigether with all necessary imports within an xml tag <UNITTEST_CLASS> 
- include docstrings of all functions, classes and modules
"""

# generate unit test

In [23]:
a = m.generate_text(prompt)

# extract unit test code

In [24]:
f_start = a.find('<UNITTEST_CLASS>')+17
f_end = a.find('</UNITTEST_CLASS>')

unittest_src = a[f_start:f_end]
print(unittest_src)

import unittest
from modules.misc import change_money

class TestChangeMoney(unittest.TestCase):
    """
    Unit tests for the change_money function.
    """

    def test_valid_input(self):
        """
        Test cases with valid input (1-99 cents).
        """
        self.assertEqual(change_money(1), [1, 0, 0])
        self.assertEqual(change_money(10), [0, 1, 0])
        self.assertEqual(change_money(25), [0, 0, 1])
        self.assertEqual(change_money(26), [1, 0, 1])
        self.assertEqual(change_money(35), [0, 1, 1])
        self.assertEqual(change_money(50), [0, 0, 2])
        self.assertEqual(change_money(61), [1, 1, 2])
        self.assertEqual(change_money(99), [4, 2, 3])
        self.assertEqual(change_money(75), [0, 0, 3])
        self.assertEqual(change_money(76), [1, 0, 3] )
        self.assertEqual(change_money(85), [0, 1, 3])
        self.assertEqual(change_money(95), [0, 2, 3])
        self.assertEqual(change_money(4), [4, 0, 0])
        self.assertEqual(change_m

# run the test

In [25]:
if not 'unittest.main()' in unittest_src:
    unittest_src += """
if __name__ == '__main__':
    unittest.main()
    """

with open('file_with_tests.py', 'w') as f:
    f.write(unittest_src)

In [26]:
command = [sys.executable, 'file_with_tests.py']

In [27]:
result = subprocess.run(
    command,
    capture_output=True,
    text=True,
    check=False, # Recommended to automatically check for errors
    encoding='utf-8'
)

In [28]:
print(result.stdout)

In [29]:
print(result.stderr)

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK



# run test smells

- https://github.com/maxpacs98/disertation

In [30]:
smells_functions = {i: eval(f'smells.{i}') for i in dir(smells) if i.startswith('check')}

{k: "ok" if len(v(unittest_src))==0 else 'not_ok' for k,v in smells_functions.items()}

{'check_assertion_roulette': 'ok',
 'check_conditional_logic': 'ok',
 'check_duplicate_assert': 'ok',
 'check_eager_test': 'ok',
 'check_exception_handling': 'ok',
 'check_ignored_test': 'ok',
 'check_magic_number': 'ok',
 'check_redundant_print': 'ok',
 'check_sleepy_test': 'ok',
 'check_unknown_test': 'ok'}

# run `pylint`

In [31]:
pylint_executable = "/".join(sys.executable.split('/')[:-1] + ['pylint'])
command = [pylint_executable, 'file_with_tests.py']

In [32]:

    result = subprocess.run(
        command,
        capture_output=True,
        text=True,
        check=False, # Recommended to automatically check for errors
        encoding='utf-8'
    )

In [33]:
print(result.stdout)

************* Module file_with_tests
file_with_tests.py:27:0: C0303: Trailing whitespace (trailing-whitespace)
file_with_tests.py:1:0: C0114: Missing module docstring (missing-module-docstring)

------------------------------------------------------------------
Your code has been rated at 9.29/10 (previous run: 9.62/10, -0.33)




In [34]:
print(result.stderr)